In [1]:
import itertools

from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import HoverTool

In [2]:
# figure units
fig_size = (500, 500)
# needs consistent units
# Wafer data
# width and heigth are calculated from the diameter
dia = 7

wafer_data = {
    'names': ['WFR001',],
    'name_x': [0,],
    'name_y': [dia/2,],
    'x': [0,],
    'y': [0,],
    'dia': [dia,],
}
wafer_source = ColumnDataSource(data=wafer_data)

In [3]:
# cell position data from
# x_coord, y_coord -> bottom left
# x_size, y_size -> width and height
# name, x, y, width, height
# colour of heat map
raw_data_1 = [['cell1', 2, 1, 0.8, 0.5], 
              ['cell2', -2, -1, 0.8, 0.5],]
colour_1 = ["#B3DE69"] * 2

raw_data_2 = [['cell4', -2, 1.5, 0.6, 0.6], 
              ['cell5', 2, -1.5, 0.6, 0.6],]
colour_2 = ["#A3AEAA"] * 2

raw_data_in = (raw_data_1, raw_data_2)
colour_in = (colour_1, colour_2)

In [4]:
cell_data = {
    'names': [],
    'name_x': [],
    'name_y': [],
    'top': [],
    'bottom': [],
    'left': [],
    'right': [],
    'colour': [],    
}

raw_data = list(itertools.chain(*raw_data_in))
colour = list(itertools.chain(*colour_in))

for i, r in enumerate(raw_data):
  cell_data['names'].append(r[0])   
  cell_data['name_x'].append(r[1] + r[4] / 2) 
  cell_data['name_y'].append(r[2] + r[3] / 2) 
  cell_data['top'].append(r[2] + r[4])   
  cell_data['bottom'].append(r[2])   
  cell_data['left'].append(r[1])   
  cell_data['right'].append(r[1] + r[3])
  cell_data['colour'].append(colour[i])

# cell_data = {
#     'top': [2, 3, 4],
#     'bottom': [1, 2, 3],
#     'left': [1, 2, 3],
#     'right': [1.2, 2.5, 3.7],
# }

cell_source = ColumnDataSource(data=cell_data)

In [5]:
# need to consolidate data into ona columndatasource
# hover tooltip to show data for a given cell
hover = HoverTool(tooltips=[
    ('Name', '@names'),
    ('X', '@left'),
    ('Y', '@bottom'),
])

plot = figure(plot_width=fig_size[0], plot_height=[1], 
              tools=[hover])

# center marker
plot.cross(x='x', y='y', size=10,
           color="#E6550D", line_width=2, source=wafer_source)

# wafer outline
plot.ellipse(x='x', y='y', width='dia', height='dia',
             color="#386CB0", fill_color=None, line_width=2, source=wafer_source)

GlyphRenderer(id='b97e8025-7887-4c2e-8861-582ce3dd86e6', ...)

In [6]:
# adding cells
# plot.quad(top=cell_data['top'], bottom=cell_data['bottom'], 
#           left=cell_data['left'], right=cell_data['right'], 
#           color="#B3DE69")

# reference https://bokeh.pydata.org/en/latest/docs/reference/plotting.html#bokeh.plotting.figure.Figure.quad
cell = plot.quad(top='top', bottom='bottom',
                 left='left', right='right',
                 fill_color='colour', 
                 source=cell_source)

In [7]:
# reference http://bokeh.pydata.org/en/latest/docs/user_guide/annotations.html#labels
wfr_label = LabelSet(x='name_x', y='name_y', text='names', level='glyph',
                     source=wafer_source, render_mode='canvas')
cell_labels = LabelSet(x='name_x', y='name_y', text='names', level='glyph',
                       source=cell_source, render_mode='canvas')

plot.add_layout(wfr_label)
plot.add_layout(cell_labels)

In [8]:
show(plot)